# Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import math
import datetime as dt
import json
import requests
from tqdm import tqdm
import os

In [2]:
import geopandas as gpd
import geojson
import h3
import osm2geojson
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString
from geopy.distance import geodesic, great_circle

In [3]:
import plotly.graph_objects as go
import plotly.express as px

In [4]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [5]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [6]:
import warnings
warnings.simplefilter('ignore')

# df_inn

In [7]:
regions = [
    '50', # Москва
    '23', # Краснодар
    '66', # Екатеринбург
    '61', # Ростов-на-Дону
    '02', # Уфа
    '16', # Казань
    '72', # Тюмень
    '74', # Челябинск
    '52', # Нижний Новгород
    '63', # Самара
    '24', # Красноярск
    '26', # Ставрополь
    '54', # Новосибирск
    '42', # Кемерово
    '59', # Пермь
    '34', # Волгоград
    '64', # Саратов
    '38', # Иркутск
    '22', # Барнаул
    '36', # Воронеж
    '56', # Оренбург
    '55', # Омск
    '25', # Владивосток
    '47', # Санкт-Петербург
    '86', # Ханты-Мансийск
    '31', # Белгород
    '18', # Ижевск
    '71', # Тула
    '33', # Владимир
    '58', # Пенза
    '27', # Хабаровск
    '69', # Тверь
    '43', # Киров
    '76', # Ярославль
    '73', # Ульяновск
    '21', # Чебоксары
    '32', # Брянск
    '35', # Вологда
    '29', # Архангельск
    '48', # Липецк
    '62', # Рязань
    '46', # Курск
    '75', # Чита
    '70', # Томск
    '68', # Тамбов
    '37', # Иваново
    '30', # Астрахань
    '40', # Калуга
    '03', # Улан-Удэ
    '39', # Калининград
    '67', # Смоленск
    '14', # Якутск
    '45', # Курган
    '11', # Сыктывкар
    '28', # Благовещенск
    '13', # Саранск
    '51', # Мурманск
    '57', # Орёл
    '12', # Йошкар-Ола
    '44', # Кострома
    '60', # Псков
    '10', # Петрозаводск
    '53', # Великий Новгород
    '89', # Салехард
    '19', # Абакан
    '65', # Южно-Сахалинск
    '41', # Петропавловск-Камчатский
    '17', # Кызыл
    '08', # Элиста
    '04', # Горно-Алтайск
    '79', # Биробиджан
    '49', # Магадан
    '87', # Анадырь
    # '83', # Нарьян-Мар
]

In [12]:
path = '/home/tochka/Documents/Other/Work/Geo/EGRUL/inn'

In [9]:
reg = '45'

In [13]:
df_inn = pd.read_csv(
    f'{path}/df_inn_{reg}_2.csv.gz',
    parse_dates=['begin_date','end_date','start_reg_date','end_reg_date'],
    dtype={
        'inn':str,
        'tax_agency_code':str,
        'end_code':str,
        'kladr_code':str,
        'kladr_11':str,
        'inn_eruz':str
    },
    sep=';',
    compression='gzip'
)

In [14]:
df_inn.head()

,inn,tax_agency_code,begin_date,end_date,end_code,full_address,kladr_code,main_okved,inn_eruz,start_reg_date,end_reg_date,year,correction_num,assets_balance,short_borrowed_funds,long_borrowed_funds,revenue,net_profit_loss,count_2020,count_2021,begin_2021,end_2021,fias_code
0,4524007730,4524,2007-05-15,2021-03-26,201,"641100, ОБЛАСТЬ КУРГАНСКАЯ, Р-Н ШУМИХИНСКИЙ, Г...",450230010000103,70.22,NaN,NaT,NaT,NaN,-1,0,0,0,0,0,1,1,0,1,NaN
1,4501212660,4500,2017-02-02,NaT,NaN,"640004, КУРГАНСКАЯ ОБЛАСТЬ, Г. КУРГАН, УЛ МОЛО...",NaN,41.20,4501212660,2019-02-19,2022-02-19,NaN,-1,10128,0,0,20523,1894,1,1,0,0,fdd65850-e1cd-421c-8608-20ff3448cdba
2,4501229416,4500,2021-02-10,NaT,NaN,"640023, КУРГАНСКАЯ ОБЛАСТЬ, Г. КУРГАН, , Д. 23",NaN,33.12,4501229416,2021-03-05,2024-03-05,NaN,-1,0,0,0,0,0,0,1,1,0,36579258
3,4510028459,4510,2013-05-28,2020-12-31,407,"641310, ОБЛАСТЬ КУРГАНСКАЯ, Р-Н КЕТОВСКИЙ, С К...",450090000010019,23.14,NaN,NaT,NaT,NaN,-1,0,0,0,0,0,1,0,0,0,NaN
4,4510005250,4500,2003-03-28,NaT,NaN,"640031, ОБЛАСТЬ КУРГАНСКАЯ, Г. КУРГАН, МКР. ТО...",450000010001408,01.13.3,NaN,NaT,NaT,NaN,-1,0,0,0,0,0,1,1,0,0,NaN


In [16]:
df_inn.loc[df_inn.fias_code.notnull(), ['inn','fias_code']]

,inn,fias_code
1,4501212660,fdd65850-e1cd-421c-8608-20ff3448cdba
2,4501229416,36579258
10,4501228892,69524716
20,4501230355,361fcb98-dae2-4812-9d32-797169749dba
24,4509007006,1d9c43fc-33ae-4173-9b52-ae6fa983e417
...,...,...
12934,4501057735,9907670c-4366-4965-ab57-a156eb671e8f
12947,4501083421,7eabaafe-d07b-4a05-aa67-cc5e5102a571
12948,4501168273,451ace87-7b52-47c2-acde-ca84dfd97b32
12954,4501175947,2368230c-c940-4454-9ecd-7e7fc6eeb909


In [17]:
df_inn.loc[df_inn.fias_code.notnull(), ['inn','fias_code']].head(100).to_excel('df_inn_fias_45.xlsx')